In [ ]:
import nltk
groucho_grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> N | Det N
VP -> V NP | VP PP | AVP NP
AVP -> AV V
AV -> 'will'
Det -> 'the'
N -> 'fine' | 'end' | 'Everything'
V -> 'be'
P -> 'in'
""")
parser = nltk.ChartParser(groucho_grammar)

sentence = ['Everything','will','be','fine','in','the','end']
trees = list(parser.parse(sentence))
t = trees[0]
print(t)

In [ ]:
import textwrap
from graphviz import Digraph, nohtml, Source

# generate dot source code
#define a class to represent each nodes
class Node(object):
    def __init__(self,n,value,leaf=False):
        self.n = n
        self.value = value
        self.leaf = leaf
    def __str__(self):
        return str(self.value)+str(self.n)
    def isLeaf(self):
        return self.leaf

class graph(object):
    def __init__(self,tree):
        self.tree = tree
        self.dot_header = [textwrap.dedent("""
                            digraph {
                            node [shape=circle, fontsize=12, fontname="Courier", height=.1];
                            ranksep=.3;
                            edge [arrowsize=.5]
                            """)]
        self.dot_body = []
        self.n_count = 0
        self.dot_footer = ['}']
        self.g = ""
    
    #replace each node label with Node object in abstract syntax tree
    #using depth first search to iterate each node
    def re_node(self,tree):
        s = '  node{} [label="{}"]\n'.format(self.n_count, tree.label())
        self.dot_body.append(s)
        tree.set_label(Node(self.n_count,tree.label()))
        self.n_count += 1
        for item in tree:
            if not isinstance(item,str):
                self.re_node(item)
            else:
                s = '  node{} [label="{}"]\n'.format(self.n_count, item)
                self.dot_body.append(s)
                tree[0] = Node(self.n_count,item,True)
                self.n_count += 1
        
    #navigate each edges in ast recursively using dfs
    def dfs(self,tree):
        src = tree.label()
        for item in tree:
            if not isinstance(item,Node):
                dest = item.label()
                s = '  node{} -> node{}\n'.format(src.n, dest.n)
                self.dot_body.append(s)
                self.dfs(item)
            else:
                src = tree.label()
                dest = item
                self.n_count += 1
                s = '  node{} -> node{}\n'.format(src.n, dest.n)
                self.dot_body.append(s)
                
    def draw(self):
        self.re_node(self.tree)
        self.dfs(self.tree)
        g = "".join(self.dot_header + self.dot_body + self.dot_footer)
        return g

src = graph(t).draw()
#see the generated DOT source code
print(src)

In [ ]:
#graph
g = Source(src)
g

In [ ]:
#generate png image for each word in the sentence
lines = src.split("\n")
for i in range(len(sentence)):
    test = src.replace(sentence[i]+'"]',sentence[i]+'" color=pink style=filled]')
    g = Source(test,filename=str(i))
    g.render(format='png')

In [ ]:
from ipywidgets import *
from IPython.display import Image
import time

#display generated png image using slidebar
def func(x):
    for i in range(len(sentence)):
        if i==x:
            print("\x1b[31m"+sentence[i]+"\x1b[0m", end=' ')
        else:
            print(sentence[i], end = ' ')
    print()
    display(Image(filename=str(x)+'.png'))

interact(func, x=widgets.IntSlider(min=0, max=len(sentence)-1, step=1));